In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# Sub-Agent Pattern Demo

Demonstrates creating specialized sub-agents (fruit & veggie experts) wrapped as tools for a parent agent with checkpointing. Fruit agent has an interrupt, and we will time travel to before the interrupt.

In [2]:
from langchain.agents import create_agent
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import Command, interrupt

@tool
def fruit_info(fruit_name: str) -> str:
    """Look up fruit info."""
    interrupt("continue?")  
    return f"Info about {fruit_name}"

@tool
def veggie_info(veggie_name: str) -> str:
    """Look up veggie info."""
    return f"Info about {veggie_name}"

# Subagents — no checkpointer setting (inherits parent)
fruit_agent = create_agent(
    model="gpt-4.1-mini",
    tools=[fruit_info],
    system_prompt="You are a fruit expert. Use the fruit_info tool. Respond in one sentence.",
)

veggie_agent = create_agent(
    model="gpt-4.1-mini",
    tools=[veggie_info],
    system_prompt="You are a veggie expert. Use the veggie_info tool. Respond in one sentence.",
)

# Wrap subagents as tools for the outer agent
@tool
def ask_fruit_expert(question: str) -> str:
    """Ask the fruit expert. Use for ALL fruit questions."""
    response = fruit_agent.invoke(
        {"messages": [{"role": "user", "content": question}]},
    )
    return response["messages"][-1].content

@tool
def ask_veggie_expert(question: str) -> str:
    """Ask the veggie expert. Use for ALL veggie questions."""
    response = veggie_agent.invoke(
        {"messages": [{"role": "user", "content": question}]},
    )
    return response["messages"][-1].content

# Outer agent with checkpointer
agent = create_agent(
    model="gpt-4.1-mini",
    tools=[ask_fruit_expert, ask_veggie_expert],
    system_prompt=(
        "You have two experts: ask_fruit_expert and ask_veggie_expert. "
        "ALWAYS delegate questions to the appropriate expert."
    ),
    checkpointer=MemorySaver(),
)

## Create Sub-Agents and Outer Agent

1. **Sub-agents**: Specialized agents (fruit/veggie experts) without checkpointers
2. **Tool wrappers**: Convert sub-agents into callable tools
3. **Outer agent**: Coordinates sub-agents using MemorySaver for state persistence

In [3]:
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "Tell me about apples"}]},
    config=config,
)


In [4]:
from pprint import pprint
pprint(response)

{'__interrupt__': [Interrupt(value='continue?',
                             id='34895816f3f8b527a9edbfd045726371')],
 'messages': [HumanMessage(content='Tell me about apples', additional_kwargs={}, response_metadata={}, id='aa1639b1-88fb-4ede-8f31-fb39887088f5'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 110, 'total_tokens': 131, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_a391f2cee0', 'id': 'chatcmpl-DCaF5nxrZfZmIEfj9S0YW3F2gNSw2', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c8cf8-8769-7741-b45c-f314e38fa904-0', tool_calls=[{'name': 'ask_fruit_expert', 'args': {'question': 'Tell

In [5]:

# Resume — approve the interrupt
response = agent.invoke(Command(resume=True), config=config)  


In [6]:
states = list(agent.get_state_history(config))

In [7]:
for state in states:
    print(state)
    print("\n------------------")

StateSnapshot(values={'messages': [HumanMessage(content='Tell me about apples', additional_kwargs={}, response_metadata={}, id='aa1639b1-88fb-4ede-8f31-fb39887088f5'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 110, 'total_tokens': 131, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_a391f2cee0', 'id': 'chatcmpl-DCaF5nxrZfZmIEfj9S0YW3F2gNSw2', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c8cf8-8769-7741-b45c-f314e38fa904-0', tool_calls=[{'name': 'ask_fruit_expert', 'args': {'question': 'Tell me about apples.'}, 'id': 'call_D57tCqoIEC8oezZTxcfmJvbW', 'type': 'tool_call'}], invalid_tool_calls=[], usage

## Select the state just prior to the interrupt

In [8]:
selected_state = states[2]
print(selected_state.interrupts)
print(selected_state.values)
print(selected_state.config)

(Interrupt(value='continue?', id='34895816f3f8b527a9edbfd045726371'),)
{'messages': [HumanMessage(content='Tell me about apples', additional_kwargs={}, response_metadata={}, id='aa1639b1-88fb-4ede-8f31-fb39887088f5'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 110, 'total_tokens': 131, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_a391f2cee0', 'id': 'chatcmpl-DCaF5nxrZfZmIEfj9S0YW3F2gNSw2', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c8cf8-8769-7741-b45c-f314e38fa904-0', tool_calls=[{'name': 'ask_fruit_expert', 'args': {'question': 'Tell me about apples.'}, 'id': 'call_D57tCqoIEC8oezZTxcfmJvbW', '

In [9]:
new_config = agent.update_state(selected_state.config, values=selected_state.values)

In [10]:
state = agent.invoke(None, new_config)

In [11]:
print(state["__interrupt__"])

[Interrupt(value='continue?', id='9a2127e66b6ef9799103b87a5264cc55')]


In [24]:
agent.invoke(Command(resume=True), config=new_config)

{'messages': [HumanMessage(content='Tell me about apples', additional_kwargs={}, response_metadata={}, id='28292354-0356-4ae8-a7e8-c84ed0fc26d9'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 110, 'total_tokens': 131, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_a391f2cee0', 'id': 'chatcmpl-DCZSCd3BwkoQMsL43587OlcsoOFbJ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c8cca-42b3-7a93-8840-5901901d6bdd-0', tool_calls=[{'name': 'ask_fruit_expert', 'args': {'question': 'Tell me about apples.'}, 'id': 'call_KSet1eDAqAVT7wORtROH8uug', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_t